In [49]:
import itertools
import copy
import math

In [9]:
form = {
    "level": {"500", "850"},
    "time": {"12:00", "00:00"},
    "param": {"Z", "T"},
    "stat": {"daily_mean", "hourly"},
    "number": {"1", "2", "3"},
    "model": {"a", "b", "c"}
}

selection = {
    "param": {"Z", "T"},
    "level": {"500", "850"},
    "stat": {"daily_mean", "hourly"},
    "time": {"12:00", "00:00"},
    "number": {"1", "2"},
    "model": {"a"}
}


constraints = [
        {"level": {"500"}, "param": {"Z", "T"}, "time": {"12:00", "00:00"}, "stat": {"hourly"}},
        {"level": {"850"}, "param": {"T"}, "time": {"12:00"}, "stat": {"hourly"}},
        {"level": {"500"}, "param": {"Z", "T"}, "stat": {"daily_mean"}},
        {"level": {"850"}, "param": {"T"}, "stat": {"daily_mean"}},
    ]
# some missing combinations:
# - every combination that includes "850" and "Z"
# - time "00:00" is not available for param "T" and level "850"


In [10]:
def get_always_valid_params(form, constraints):
    result={}
    for field_name, field_values in form.items():
        if field_name not in get_keys(constraints):
            result.setdefault(field_name, field_values)
    return result

In [11]:
def get_keys(constraints):
    keys = set()
    for constraint in constraints:
        keys |= set(constraint.keys())
    return keys

In [43]:
always_valid = get_always_valid_params(form, constraints)
always_valid

dict_values([{'1', '3', '2'}, {'b', 'a', 'c'}])

In [52]:
math.prod(map(len, always_valid.values()))

9

In [17]:
sub_selection = {k:v for k,v in selection.items() if k not in always_valid}
sub_selection

{'param': {'T', 'Z'},
 'level': {'500', '850'},
 'stat': {'daily_mean', 'hourly'},
 'time': {'00:00', '12:00'}}

In [20]:
def compute_layers(d):
    keys, values = zip(*d.items())
    return [dict(zip(keys, v)) for v in itertools.product(*values)]

layers = compute_layers(sub_selection)
layers

[{'param': 'T', 'level': '500', 'stat': 'daily_mean', 'time': '12:00'},
 {'param': 'T', 'level': '500', 'stat': 'daily_mean', 'time': '00:00'},
 {'param': 'T', 'level': '500', 'stat': 'hourly', 'time': '12:00'},
 {'param': 'T', 'level': '500', 'stat': 'hourly', 'time': '00:00'},
 {'param': 'T', 'level': '850', 'stat': 'daily_mean', 'time': '12:00'},
 {'param': 'T', 'level': '850', 'stat': 'daily_mean', 'time': '00:00'},
 {'param': 'T', 'level': '850', 'stat': 'hourly', 'time': '12:00'},
 {'param': 'T', 'level': '850', 'stat': 'hourly', 'time': '00:00'},
 {'param': 'Z', 'level': '500', 'stat': 'daily_mean', 'time': '12:00'},
 {'param': 'Z', 'level': '500', 'stat': 'daily_mean', 'time': '00:00'},
 {'param': 'Z', 'level': '500', 'stat': 'hourly', 'time': '12:00'},
 {'param': 'Z', 'level': '500', 'stat': 'hourly', 'time': '00:00'},
 {'param': 'Z', 'level': '850', 'stat': 'daily_mean', 'time': '12:00'},
 {'param': 'Z', 'level': '850', 'stat': 'daily_mean', 'time': '00:00'},
 {'param': 'Z', 

In [56]:
def check_layer_avalability(layer, constraints):
    print(layer)
    for constraint in constraints:
        ok = True;
        for key, value in layer.items():
            if key not in constraint.keys():
                diff = set(layer) - set(constraint.keys())
                if diff:
                    for key in diff:
                        layer.pop(key)
                        if check_layer_avalability(layer, constraints):
                            #print(True)
                            return True
                ok = False;
                break
            elif value not in constraint[key]:
                ok = False;
                break
        if ok:
            print(True)
            return True
    print(ok)
    return ok

In [57]:
def count_available(layers, constaints, always_valid):
    layers = copy.deepcopy(layers)
    count = 0
    for layer in layers:
        if check_layer_avalability(layer, constaints):
            count += 1
    return count * math.prod(map(len, always_valid.values()))
    

In [58]:
count_available(layers, constraints, always_valid)

{'param': 'T', 'level': '500', 'stat': 'daily_mean', 'time': '12:00'}
{'param': 'T', 'level': '500', 'stat': 'daily_mean'}
True
{'param': 'T', 'level': '500', 'stat': 'daily_mean', 'time': '00:00'}
{'param': 'T', 'level': '500', 'stat': 'daily_mean'}
True
{'param': 'T', 'level': '500', 'stat': 'hourly', 'time': '12:00'}
True
{'param': 'T', 'level': '500', 'stat': 'hourly', 'time': '00:00'}
True
{'param': 'T', 'level': '850', 'stat': 'daily_mean', 'time': '12:00'}
{'param': 'T', 'level': '850', 'stat': 'daily_mean'}
True
{'param': 'T', 'level': '850', 'stat': 'daily_mean', 'time': '00:00'}
{'param': 'T', 'level': '850', 'stat': 'daily_mean'}
True
{'param': 'T', 'level': '850', 'stat': 'hourly', 'time': '12:00'}
True
{'param': 'T', 'level': '850', 'stat': 'hourly', 'time': '00:00'}
False
{'param': 'Z', 'level': '500', 'stat': 'daily_mean', 'time': '12:00'}
{'param': 'Z', 'level': '500', 'stat': 'daily_mean'}
True
{'param': 'Z', 'level': '500', 'stat': 'daily_mean', 'time': '00:00'}
{'par

99